**Exercise 8** 
This is the code to compute the likelihood function of exercise 8 of problem set 3. 


In [1]:
using StatFiles, DataFrames

data_path = "/Users/jorditorresvallverdu/Downloads/xsection_05.dta"

export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/policy_effects/figures/"


main = DataFrame(load(data_path))

Row,occup,d_months,hwage
,Float32?,Float32?,Float32?
1,3.0,10.0,47.619
2,2.0,1.0,18.4854
3,2.0,7.0,22.5694
4,3.0,244.0,21.3294
5,3.0,88.0,7.93651
6,3.0,124.0,16.7143
7,3.0,136.0,21.645
8,1.0,0.0,missing
9,3.0,2.0,25.5952


In [10]:
##Code tbe parameters of the likelihood function
using Distributions

function log_likelihood(θ, w_i_star,w_f_star,t_u, w_i, w_f,u, e_i, e_f)

μ_i, μ_f, σ_i, σ_f, λ_i, λ_f, η_i, η_f = unpack_params(θ)


Fi_star= cdf.(LogNormal(μ_i, σ_i), w_i_star) #need to define w_i_star properly following Heckman
Ff_star= cdf.(LogNormal(μ_f, σ_f), w_f_star) #need to define w_f_star outside too

F_i_tilde=  1-Fi_star
F_f_tilde=  1-Ff_star

a_i=  λ_i* F_i_tilde
a_f=  λ_f* F_f_tilde

h_u=  a_i + a_f

p_u= (1+(a_i/η_i)+(a_f/η_f))^-1
p_ei= (a_i/η_i)/(1+(a_i/η_i)+(a_f/η_f))
p_ef=  (a_f/η_f)/(1+(a_i/η_i)+(a_f/η_f))

f_i= pdf.(LogNormal(μ_i, σ_i), w_i) ##need to define w_i as input
f_f= pdf.(LogNormal(μ_f, σ_f), w_f)

T = eltype(θ)
f_i_full = zeros(T, length(u))
f_f_full = zeros(T, length(u))

f_i_full[e_i] .= f_i
f_f_full[e_f] .= f_f


ll_u= @. u*(log(h_u) - h_u* t_u+ log(p_u))
ll_ei= @. e_i *(log(η_i)+(-η_i*t_u) + log(f_i_full)- log(F_i_tilde)+ log(p_ei)) 
ll_ef= @. e_f*(log(η_f)+(-η_f*t_u) + (log(f_f_full))- log(F_f_tilde)+ log(p_ef))


    return -(sum(ll_u) + sum(ll_ef) + sum(ll_ei))
end


log_likelihood (generic function with 1 method)

In [3]:
##define parameters/inputs of the function 
u= (main.occup .== 1) #vector
e_f= (main.occup .==2) #vector
e_i= (main.occup .==3) #vector

t_u=main.d_months #vector


w_i=main.hwage[e_i] #vector
w_f=main.hwage[e_f] #vector 

w_i_star= minimum(w_i) #scalar
w_f_star= minimum(w_f) #scalar



5.7963266f0

In [4]:
function unpack_params(θ::AbstractVector{T}) where T
    μ_i = θ[1]
    μ_f = θ[2]

    σ_i = exp(θ[3])
    σ_f = exp(θ[4])

    λ_i = exp(θ[5])
    λ_f = exp(θ[6])

    η_i = exp(θ[7])
    η_f = exp(θ[8])

    return μ_i, μ_f, σ_i, σ_f, λ_i, λ_f, η_i, η_f
end



unpack_params (generic function with 1 method)

In [5]:
θ0 = [
    mean(log.(w_i)),             # μ_i
    mean(log.(w_f)),             # μ_f
    log(std(log.(w_i))),         # log σ_i
    log(std(log.(w_f))),         # log σ_f
    log(0.2),                    # log λ_i
    log(0.2),                    # log λ_f
    log(0.05),                   # log η_i
    log(0.05)                    # log η_f
]

obj(θ) = log_likelihood(θ,
    w_i_star, w_f_star,
    t_u, w_i, w_f,
    u, e_i, e_f
)

obj (generic function with 1 method)

In [11]:
##Optimize
using Optim

res = optimize(obj, θ0, BFGS())
θ_hat = Optim.minimizer(res)
LL_min = Optim.minimum(res)

println("Estimated parameters θ̂ = ", θ_hat)
println("Minimum negative log-likelihood = ", LL_min)



Estimated parameters θ̂ = [2.8804903767235714, 3.136134285968575, -0.5454886682843559, -0.6474618104017853, -2.0161635002976817, -1.8557317508455138, -4.598597454329567, -4.570168380212041]
Minimum negative log-likelihood = 150136.15985337266


In [14]:
using FiniteDiff, LinearAlgebra

H_hat = FiniteDiff.finite_difference_hessian(obj, θ_hat)

V = inv(H_hat)
se = sqrt.(diag(V))

println("Standard errors = ", se)


Standard errors = [0.007088607541782665, 0.006088249077972343, 0.008851085822646178, 0.008676421160907886, 0.034093724885267546, 0.0335001287505111, 0.012088870912597303, 0.01132223341160289]


In [15]:
using DataFrames

varnames=["μ_i", "μ_f", "σ_i", "σ_f", "λ_i", "λ_f", "η_i", "η_f"]

results_df = DataFrame(
        term = ["β$i ($name)" for (i, name) in enumerate(varnames)], ##quite nice tool to actually loop in a more efficient way over order of variables and variable names. 
        raw_estimate = θ_hat,
        exp_estimate=exp.(θ_hat) ,
        std_error = se
    )



Row,term,raw_estimate,exp_estimate,std_error
,String,Float64,Float64,Float64
1,β1 (μ_i),2.88049,17.823,0.00708861
2,β2 (μ_f),3.13613,23.0147,0.00608825
3,β3 (σ_i),-0.545489,0.579559,0.00885109
4,β4 (σ_f),-0.647462,0.523373,0.00867642
5,β5 (λ_i),-2.01616,0.133165,0.0340937
6,β6 (λ_f),-1.85573,0.156338,0.0335001
7,β7 (η_i),-4.5986,0.0100659,0.0120889
8,β8 (η_f),-4.57017,0.0103562,0.0113222


In [16]:
#this is a function to recover the wages from the parameters
using QuadGK, Distributions

function tail_integral_numeric(wstar, μ, σ)
    dist = LogNormal(μ, σ)
    integrand(w) = (w - wstar) * pdf(dist, w)
    val, _ = quadgk(integrand, wstar, Inf)
    return val
end

ρ=0.05
B_i= 1.92
B_f= 3

function get_b(w_star, B_j, λ_j, η_j, μ_j, σ_j)   
    return b = w_star + B_j - (λ_j / (ρ + η_j)) * tail_integral_numeric(w_star, μ_j, σ_j)

end 



get_b (generic function with 1 method)

In [17]:
##apply the actual function

b_i= get_b(w_i_star, B_i, results_df.exp_estimate[5], results_df.exp_estimate[7], results_df.raw_estimate[1], results_df.exp_estimate[3])

-35.2457585604605

In [18]:
b_f= get_b(w_f_star, B_f, results_df.exp_estimate[6], results_df.exp_estimate[8], results_df.raw_estimate[2], results_df.exp_estimate[4])

-44.563062802559514

**2. Generate counterfactuals** 

In [21]:
using StatFiles, DataFrames

data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year2/term1/bobba/data/panel_06.dta"


main_2 = DataFrame(load(data_path))


Row,occup,sp,educ,age,cvemun,sec_occup,quarter,hwage,pobtot,indmarg95
,Float32?,Int8?,Float32?,Int8?,Int32?,Float32?,Float32?,Float32?,Float64?,Float32?
1,2.0,1,1.0,51,1001,0.0,3.0,35.8245,643419.0,-0.37098
2,2.0,1,0.0,35,1001,0.0,1.0,13.2479,643419.0,-0.37098
3,2.0,1,0.0,46,1001,0.0,4.0,23.1068,643419.0,-0.37098
4,2.0,1,1.0,51,1001,0.0,4.0,38.2128,643419.0,-0.37098
5,2.0,1,0.0,38,1001,0.0,4.0,16.121,643419.0,-0.37098
6,2.0,1,1.0,34,1001,0.0,4.0,45.1389,643419.0,-0.37098
7,2.0,1,0.0,37,1001,0.0,4.0,18.4855,643419.0,-0.37098
8,2.0,1,1.0,46,1001,0.0,2.0,69.3204,643419.0,-0.37098
9,1.0,1,0.0,26,1001,missing,2.0,missing,643419.0,-0.37098


In [30]:
main_2.b_i_c = ifelse.(main_2.sp .== 1, 2.84, 2.13)
main_2.b_f_c .= 3.0

5440-element Vector{Float64}:
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 ⋮
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0

In [ ]:
#get wages function-->find Fixed Point 

using Roots


# residual of the reservation wage equation: should be == 0 at wstar = w_j^*
function wstar_residual(wstar, b, B_j, λ_j, η_j, μ_j, σ_j)
    rhs = b - B_j + (λ_j / (ρ + η_j)) * tail_integral_numeric(wstar, μ_j, σ_j)
    return wstar - rhs
end

function get_wstar(b, B_j, λ_j, η_j, μ_j, σ_j; initial_guess = exp(μ_j))
    f(w) = wstar_residual(w, b, B_j, λ_j, η_j, μ_j, σ_j)
    return find_zero(f, initial_guess) ##from Roots, f(x)=0, it finds where the function crosses 0
end



get_wstar (generic function with 1 method)

In [ ]:
#This is just to generate the truncation point

b_i_c=2.13 ##new bi_c we want to introduce (this is the key point of using 2005 data to see something on 2006; we assume that what changes from 2005->2006 is just this (strong assumption))
b_i_t=2.84
b_f_c=3

w_i_star_cf_c= get_wstar(b_i,b_i_c, results_df.exp_estimate[5], results_df.exp_estimate[7], results_df.raw_estimate[1], results_df.exp_estimate[3])

w_i_star_cf_t = get_wstar(b_i,b_i_t, results_df.exp_estimate[5], results_df.exp_estimate[7], results_df.raw_estimate[1], results_df.exp_estimate[3])

w_f_star_cf = get_wstar(b_f, b_f_c, results_df.exp_estimate[6], results_df.exp_estimate[8], results_df.raw_estimate[2], results_df.exp_estimate[4])


5.796326637268066

In [ ]:
using Distributions, Random

dist_i = LogNormal(results_df.raw_estimate[1], results_df.exp_estimate[3])
dist_f = LogNormal(results_df.raw_estimate[2], results_df.exp_estimate[4])

F_i_star_control = cdf(dist_i, w_i_star_cf_c) #this is what changes, truncation
F_i_star_treated = cdf(dist_i, w_i_star_cf_t)
F_f_star         = cdf(dist_f, w_f_star)


0.004211048094872542

In [ ]:
#Randomization
n = nrow(main2)
main2.hwage_cf = similar(main2.hwage)  # same type/length as original

Random.seed!(12345)  # for reproducibility

for i in 1:n
    occ = main2.occup[i]
    sp  = main2.sp[i]

    if occ == 1
        #unemployed
        main2.hwage_cf[i] = NaN
    elseif occ == 2
        #formal job 
        u  = rand()
        p  = F_f_star + u * (1 - F_f_star) #F(w∣w>wf∗​), this comes from the definition of F_truncated...
        w  = quantile(dist_f, p) #w=F^{−1}(p)
        main2.hwage_cf[i] = w
    elseif occ == 3
        # informal job
        if sp == 1
            #treated municipality
            u  = rand()
            p  = F_i_star_treated + u * (1 - F_i_star_treated)
            w  = quantile(dist_i, p)
            main2.hwage_cf[i] = w
        else
            #control municipality
            u  = rand()
            p  = F_i_star_control + u * (1 - F_i_star_control)
            w  = quantile(dist_i, p)
            main2.hwage_cf[i] = w
        end
    
        
    end
end

main2.lhwage_cf = log.(main2.hwage_cf)


5440-element Vector{Float32}:
   3.9726932
   3.719403
   2.923675
   2.570147
   2.8991623
   2.7924511
   3.3557189
   3.4565144
 NaN
   2.8782122
   ⋮
   3.499987
   3.413409
   3.5073833
   2.4215848
   2.0935817
   3.356275
   4.080357
   3.8398993
   2.4836686

In [ ]:
using DataFrames, FixedEffectModels, Statistics

data_fe = dropmissing(main2, :lhwage_cf)


data_fe = filter(
    row -> isfinite(row.lhwage_cf),
    data_fe
)
twfe_sim = reg(
    data_fe,
    @formula(lhwage_cf ~ sp + fe(cvemun) + fe(quarter))
)

println(twfe_sim)
#add cluster se!!!!


                         FixedEffectModel                         
Number of obs:              5284  Converged:                  true
dof (model):                   1  dof (residuals):            5085
R²:                        0.037  R² adjusted:              -0.000
F-statistic:            0.775633  P-value:                   0.379
R² within:                 0.000  Iterations:                    4
     Estimate  Std. Error  t-stat  Pr(>|t|)   Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────
sp  0.0487816   0.0553895  0.8807    0.3785  -0.0598058   0.157369

